- Paulina Muñoz
- Jerónimo Bedoya

# Tarea modelos de ensamble

Se le entrega un archivo con datos de canciones en Spotify. Usando este archivo haga un modelo de ensamble que prediga la popularidad de las canciones.

El modelo de ensamble debe ser de alguno de estos tipos: XGBoost, CatBoost o LGBM.

La métrica que debe utilizar es RMSE.

La partición del dataset en datos de entrenamiento y prueba debe hacerse en una proporción 80/20, usando el parámetro random_state=72.

Debe entregar el menor RMSE de generalización encontrado, los hiperparámetros sintonizados, el error de prueba, y un dataframe con la importancia de cada característica.

Sí el RMSE de prueba está por encima de 23, la máxima bonificación que obtendrá es 0.3.

Sí el RMSE de prueba está por encima de 20, la máxima bonificación que obtendrá es 0.4.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/top_10000_1960-now.csv')
df.head()

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,...,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Album Genres,Label,Copyrights
0,spotify:track:1XAZlnVtthcDZt2NI1Dtxo,Justified & Ancient - Stand by the Jams,spotify:artist:6dYrdRlNZSKaVxYg5IrvCH,The KLF,spotify:album:4MC0ZjNtVP1nDD5lsLxFjc,Songs Collection,spotify:artist:6dYrdRlNZSKaVxYg5IrvCH,The KLF,1992-08-03,https://i.scdn.co/image/ab67616d0000b27355346b...,...,0.0480,0.0158,0.112000,0.4080,0.504,111.458,4.0,NaN,Jams Communications,"C 1992 Copyright Control, P 1992 Jams Communic..."
1,spotify:track:6a8GbQIlV8HBUW3c6Uk9PH,I Know You Want Me (Calle Ocho),spotify:artist:0TnOYISbd1XYRBk9myaseg,Pitbull,spotify:album:5xLAcbvbSAlRtPXnKkggXA,Pitbull Starring In Rebelution,spotify:artist:0TnOYISbd1XYRBk9myaseg,Pitbull,2009-10-23,https://i.scdn.co/image/ab67616d0000b27326d73a...,...,0.1490,0.0142,0.000021,0.2370,0.800,127.045,4.0,NaN,Mr.305/Polo Grounds Music/J Records,"P (P) 2009 RCA/JIVE Label Group, a unit of Son..."
2,spotify:track:70XtWbcVZcpaOddJftMcVi,From the Bottom of My Broken Heart,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears,spotify:album:3WNxdumkSMGMJRhEgK80qx,...Baby One More Time (Digital Deluxe Version),spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears,1999-01-12,https://i.scdn.co/image/ab67616d0000b2738e4986...,...,0.0305,0.5600,0.000001,0.3380,0.706,74.981,4.0,NaN,Jive,P (P) 1999 Zomba Recording LLC
3,spotify:track:1NXUWyPJk5kO6DQJ5t7bDu,Apeman - 2014 Remastered Version,spotify:artist:1SQRv42e4PjEYfPhS0Tk9E,The Kinks,spotify:album:6lL6HugNEN4Vlc8sj0Zcse,"Lola vs. Powerman and the Moneygoround, Pt. On...",spotify:artist:1SQRv42e4PjEYfPhS0Tk9E,The Kinks,2014-10-20,https://i.scdn.co/image/ab67616d0000b2731e7c53...,...,0.2590,0.5680,0.000051,0.0384,0.833,75.311,4.0,NaN,Sanctuary Records,"C © 2014 Sanctuary Records Group Ltd., a BMG C..."
4,spotify:track:72WZtWs6V7uu3aMgMmEkYe,You Can't Always Get What You Want,spotify:artist:22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,spotify:album:0c78nsgqX6VfniSNWIxwoD,Let It Bleed,spotify:artist:22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,1969-12-05,https://i.scdn.co/image/ab67616d0000b27373d927...,...,0.0687,0.6750,0.000073,0.2890,0.497,85.818,4.0,NaN,Universal Music Group,"C © 2002 ABKCO Music & Records Inc., P ℗ 2002 ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Track URI             9999 non-null   object 
 1   Track Name            9998 non-null   object 
 2   Artist URI(s)         9997 non-null   object 
 3   Artist Name(s)        9998 non-null   object 
 4   Album URI             9997 non-null   object 
 5   Album Name            9998 non-null   object 
 6   Album Artist URI(s)   9997 non-null   object 
 7   Album Artist Name(s)  9997 non-null   object 
 8   Album Release Date    9997 non-null   object 
 9   Album Image URL       9995 non-null   object 
 10  Disc Number           9999 non-null   int64  
 11  Track Number          9999 non-null   int64  
 12  Track Duration (ms)   9999 non-null   int64  
 13  Track Preview URL     7102 non-null   object 
 14  Explicit              9999 non-null   bool   
 15  Popularity           

In [ ]:
df['Album Release Date'].dropna(inplace = True)
df['Album Release Date'] = pd.to_datetime(df['Album Release Date'], errors='coerce')
df['Year'] = df['Album Release Date'].dt.year

Regresión con Catboost


In [ ]:
from sklearn.model_selection import train_test_split
# df_sel = df.drop(['Album Genres','Track URI','Album Image URL','Track Preview URL'],axis = 1)
df_sel = df.drop('Album Genres',axis = 1)
df_sel.dropna(inplace=True)
df_sel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5783 entries, 1 to 9998
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Track URI             5783 non-null   object        
 1   Track Name            5783 non-null   object        
 2   Artist URI(s)         5783 non-null   object        
 3   Artist Name(s)        5783 non-null   object        
 4   Album URI             5783 non-null   object        
 5   Album Name            5783 non-null   object        
 6   Album Artist URI(s)   5783 non-null   object        
 7   Album Artist Name(s)  5783 non-null   object        
 8   Album Release Date    5783 non-null   datetime64[ns]
 9   Album Image URL       5783 non-null   object        
 10  Disc Number           5783 non-null   int64         
 11  Track Number          5783 non-null   int64         
 12  Track Duration (ms)   5783 non-null   int64         
 13  Track Preview URL     5

In [ ]:
X = df_sel.drop('Popularity', axis=1)
cat_vars = list(X.select_dtypes(include=['object']).columns)
y = df_sel['Popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=72)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
model = CatBoostRegressor(
    cat_features=cat_vars, # Codifica one-hot variables categóricas
    one_hot_max_size=10, # Limita el número de características que genera al hacer codificación one-hot. Recomendable cuando una característica tiene muchas categorías
    early_stopping_rounds=5 # Si después de 5 iteraciones la métrica no cambia, para
    )

dist = {'n_estimators':range(1, 201), 'max_depth':range(1,20), 'learning_rate':loguniform(1e-2,1e0)}
grid_search = RandomizedSearchCV(estimator=model, param_distributions=dist, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)


print(f'Best score: {-grid_search.best_score_:.2f} with {grid_search.best_params_}')
print(f'Test score: {root_mean_squared_error(y_test,grid_search.best_estimator_.predict(X_test)):.2f}')


0:	learn: 20.0589360	total: 41.1ms	remaining: 3.45s
1:	learn: 19.8698682	total: 64.9ms	remaining: 2.69s
2:	learn: 19.7256455	total: 89.4ms	remaining: 2.44s
3:	learn: 19.5561050	total: 110ms	remaining: 2.22s
4:	learn: 19.3820920	total: 132ms	remaining: 2.11s
5:	learn: 19.2141484	total: 153ms	remaining: 2.02s
6:	learn: 19.0461634	total: 177ms	remaining: 1.97s
7:	learn: 18.8957872	total: 207ms	remaining: 1.99s
8:	learn: 18.7544691	total: 232ms	remaining: 1.96s
9:	learn: 18.6146327	total: 258ms	remaining: 1.94s
10:	learn: 18.4723126	total: 281ms	remaining: 1.89s
11:	learn: 18.3434996	total: 302ms	remaining: 1.84s
12:	learn: 18.2182373	total: 325ms	remaining: 1.8s
13:	learn: 18.0941563	total: 346ms	remaining: 1.75s
14:	learn: 17.9972926	total: 369ms	remaining: 1.72s
15:	learn: 17.9024667	total: 390ms	remaining: 1.68s
16:	learn: 17.8083767	total: 441ms	remaining: 1.76s
17:	learn: 17.7190025	total: 487ms	remaining: 1.81s
18:	learn: 17.6426214	total: 539ms	remaining: 1.87s
19:	learn: 17.549082

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2395, in _fit
    tra

0:	learn: 20.1718605	total: 48.1ms	remaining: 9.34s
1:	learn: 19.9895447	total: 90.4ms	remaining: 8.72s
2:	learn: 19.8303704	total: 155ms	remaining: 9.93s
3:	learn: 19.6620144	total: 193ms	remaining: 9.23s
4:	learn: 19.5019081	total: 246ms	remaining: 9.36s
5:	learn: 19.3565383	total: 294ms	remaining: 9.26s
6:	learn: 19.2022632	total: 347ms	remaining: 9.32s
7:	learn: 19.0657777	total: 400ms	remaining: 9.36s
8:	learn: 18.9259952	total: 453ms	remaining: 9.36s
9:	learn: 18.7970965	total: 492ms	remaining: 9.11s
10:	learn: 18.6698076	total: 550ms	remaining: 9.2s
11:	learn: 18.5422146	total: 588ms	remaining: 8.97s
12:	learn: 18.4381400	total: 641ms	remaining: 8.97s
13:	learn: 18.3244460	total: 680ms	remaining: 8.79s
14:	learn: 18.2252060	total: 732ms	remaining: 8.78s
15:	learn: 18.1237562	total: 778ms	remaining: 8.71s
16:	learn: 18.0207398	total: 829ms	remaining: 8.68s
17:	learn: 17.9284853	total: 866ms	remaining: 8.52s
18:	learn: 17.8295256	total: 917ms	remaining: 8.49s
19:	learn: 17.7456302

In [ ]:
pesos = grid_search.best_estimator_.get_feature_importance(prettified=True)
pesos

,Feature Id,Importances
0,Label,16.258424
1,Artist Genres,10.811954
2,Track Duration (ms),9.007757
3,Album Release Date,6.992508
4,Year,6.024562
5,Album Artist URI(s),5.440192
6,Loudness,4.886484
7,Track Number,4.016674
8,Album URI,3.483821
9,Added At,3.226272


Podemos determinar que la variable Label es la que mas peso o importancia da al modelo, mientras que variables como Track URI, Album Image URL, Track Preview URL o incluso Artist Names no aportan nada al modelo. Se hizo la prueba excluyendo estas variables pero el Test Score queda por encima de 20.